In [15]:
from textblob import TextBlob
from attributegetter import *
from generatengrams import ngrammatch
from Contexts import *
import json
from Intents import *
import random

In [16]:
def check_actions(current_intent, attributes, active_contexts):
    '''Performs actions pertaining to current intent'''
    for action in intents.actions:
        if action.contexts_satisfied(active_contexts):
            return perform_action()

In [17]:
def check_required_params(current_intent, attributes, active_contexts):
    '''Collects attributes pertaining to the current intent'''
    
    for para in current_intent.params:
        if para.required:
            if para not in attributes:
                return random.choice(para.prompts), active_contexts
    return None, active_contexts
    

In [18]:
def input_processor(user_input, context, attributes, intent):
    '''Spellcheck and entity extraction functions go here'''
    
    uinput = TextBlob(user_input).correct().string
    
    #update the attributes, abstract over the entities in user input
    attributes, cleaned_input = getattributes(user_input, context, attributes)
    
    return attributes, cleaned_input
    

In [19]:
def loadIntent(path, intent):
    with open(path) as fil:
        dat = json.load(fil)
        intent = dat[intent]
        return Intent(intent['intentname'],intent['Parameters'])

def intentIdentifier(clean_input, context,current_intent):
    clean_input = clean_input.lower()
    scores = ngrammatch(clean_input)
    scores = sorted_by_second = sorted(scores, key=lambda tup: tup[1])
    # print clean_input
    # print 'scores', scores

    if(current_intent==None):
        if(clean_input=="search"):
            return loadIntent('data/params/params.cfg', 'FlightSearch')
        if(clean_input=='book'):
            return loadIntent('data/params/params.cfg','FlightBook')
        else:
            if scores[-1][0].startswith('book'):
                return loadIntent('data/params/params.cfg','FlightBook')
            elif scores[-1][0].startswith('search'):
                return loadIntent('data/params/params.cfg','FlightSearch')
            else:
                return loadIntent('data/params/params.cfg','FlightSearch')
    else:
        return current_intent

In [ ]:
class Session:
    def __init__(self, attributes=None, active_contexts=[]):
        
        '''Initialise a default session'''
        
        #Contexts are flags which control dialogue flow
        #self.active_contexts = active_contexts
        #self.context = FirstGreeting()
        self.context = FirstGreeting()
        
        #Intent tracks the current state of dialogue
        #self.current_intent = First_Greeting()
        self.current_intent = None
        
        #attributes hold the information collected over the conversation
        self.attributes = {}
        
    def reply(self, user_input):
        '''Generate response to user input'''
        
        self.attributes, clean_input = input_processor(user_input, self.context, self.attributes, self.current_intent)
        
        self.current_intent = intentIdentifier(clean_input, self.context, self.current_intent)
        
        prompt, self.context = check_required_params(self.current_intent, self.attributes, self.context)
        
        if prompt is not None:
            return prompt
        
        self.current_intent = intentIdentifier(clean_input, self.context, current_intent)
        prompt, self.context = check_required_parameters(self.current_intent, self.attributes, self.context)
        print self.attributes
        return prompt

In [ ]:
session = Session()

print 'BOT: sup?'

while True:
    
    inp = raw_input('User: ')
    print session.reply(inp)

BOT: sup?
User: hi
['hi']
fdsasfD
Where do you want the flight from?
User: Delhi 
['@From_LOC_delhi']
fdsasfD
Where do you want the flight from?
